# Trabajo Práctico 1
## Business Analytics / Aplicaciones Computacionales en Negocios
### Amigo, Caracoix, Marsili y Sanson

#### Setup del entorno, carga de los datos y armado de funciones básicas

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib as plt
import random
import math

from typing import List, Tuple

In [3]:
# Cargamos el dataset de viento
datos = pd.read_excel('wind_data.xlsx')

La funcion $\theta$ esta definida como:

$$
\theta(i) = 6 + 2cos(2πi/365)
$$

In [4]:
def theta_function(i):
    return 6 + 2 * np.cos(2 * np.pi * i / 365)

El error en la regresión lineal, conocido como RMSE (raiz del error cuadratico medio o root mean squared error), está definido como:

$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

In [8]:
def calcular_RMSE(Y, predicciones):

    n = len(Y)

    residuos = Y - predicciones
    varianza = np.sum(residuos**2) / n

    return np.sqrt(varianza)

##### Por último, establecemos parámetros básicos que nos van a acompañar durante todo el trabajo

In [5]:
N = 10000 # Cantidad de simulaciones/iteraciones que se haran en cada ejercicio

T = 365 # Período en el cual se realizará el análisis

# Ejercicio 1

### Regresion Lineal para el modelo del viento regional

Vamos a estimar los parámetros $\kappa$ y $\sigma$ para el regresion_vientoo de velocidad del viento regional utilizando un regresion_vientoo de regresion lineal.

La ecuación original es:

$$
S_{i+1} = S_i + \kappa (\theta(t_i) - S_i) \Delta + \sigma \sqrt{\Delta} Z_i
$$

Despejamos $S_i$ para obtener una forma adecuada para la regresión lineal:

$$
S_{i+1} - S_i = \kappa \Delta (\theta(t_i) - S_i) + \sigma \sqrt{\Delta} Z_i
$$

Reescribimos la ecuación en términos de $\beta$:

$$
Y = \beta_1 X  + \epsilon_i
$$

donde:
- $Y = S_{i+1} - S_i$
- $\beta_1 = \kappa \Delta$
- $\epsilon_i = \sigma \sqrt{\Delta} Z_i$

Luego, generamos las columnas correspndientes en nuestro dataset: 

In [6]:
datos['Y'] = datos['regional wind'].diff()  # Calcula la diferencia entre días consecutivos

# Preparar X, que es theta_i - S_i 
datos['X'] = theta_function(datos["dia"]) - datos['regional wind'].shift(1)

# Eliminar los valores nulos generados por diff() y shift()
datos = datos.dropna(subset=['Y', 'X'])

In [11]:
Y = datos['Y'] # Variable a predecir
X = datos[['X']] # Variable predictora

regresion_viento = LinearRegression()
regresion_viento.fit(X, Y)

B0 = regresion_viento.intercept_
B1 = regresion_viento.coef_[0]

delta = 1/365

#Estimamos kappa
kappa_estimado = B1 / delta

#Estimamos sigma
predicciones = regresion_viento.predict(X)

error = calcular_RMSE(Y,predicciones)

sigma_estimado = error / np.sqrt(delta)

print(f"kappa estimado: {kappa_estimado}")
print(f"sigma estimado: {sigma_estimado}")

kappa estimado: 108.73826243919804
sigma estimado: 16.046573595261595


### Regresion Lineal para el modelo de diferencia de la velocidad del viento entre el norte y el sur

Vamos a estimar los parámetros $\beta$ y $\gamma$ para el regresion_vientoo de diferencia de la velocidad del viento entre el norte y el sur utilizando un modelo de regresion lineal.

La ecuación original es:

$$
D_{i+1} = D_i - \beta D_i \Delta + \gamma \sqrt{\Delta} W_i
$$

Despejamos $D_i$ para obtener una forma adecuada para la regresión lineal:

$$
D_{i+1} - D_i = - \beta \Delta D_i  + \gamma \sqrt{\Delta} W_i
$$

Reescribimos la ecuación en términos de $\beta$:

$$
H = \beta_1 G  + \epsilon_i
$$

donde:
- $H = D_{i+1} - D_i$
- $\beta_1 = -\beta \Delta$
- $\epsilon_i = \gamma \sqrt{\Delta} W_i$

Repetimos el mismo proceso que realizamos antes:

In [12]:
# Preparar H
datos['H'] = datos['wind gap norte sur'].diff()

# Eliminar los valores nulos generados por diff()
datos = datos.dropna(subset = ('H'))

In [14]:
G = datos[['wind gap norte sur']]
H = datos['H']

regresion_gap = LinearRegression()
regresion_gap.fit(G,H)

B0_2 = regresion_gap.intercept_
B1_2 = regresion_gap.coef_[0]

#Estimamos beta
beta_estimado = -B1_2 / delta

#Estimamos gamma

predicciones_gap = regresion_gap.predict(G)

error_gap = calcular_RMSE(H,predicciones_gap)

gamma_estimado = error_gap / np.sqrt(delta)

print(f"beta estimado: {beta_estimado}")
print(f"gamma estimado: {gamma_estimado}")

beta estimado: 48.37880318325385
gamma estimado: 10.091915756839064
